# ESt ce que le salaire des pilotes de F1 n est que base sur leur performances passes.

### Problème : Peut-on deviner le salaire d'un pilote F1 basé sur ses stats ? (Poles, Victoires, Podiums, Points, etc.) (27/05/25)

### (Tensorflow)

In [3]:
from bs4 import BeautifulSoup
import requests
from io import StringIO
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


# On a trouver un nouveau endroit pour chopper toutes les data.
#### On commence part filtrer la tableau avec le nom des pilotes de la saison actuelle.
#### On decide de garder la categorie starts, wins,poles , points et podiums pour realiser notre model de ML.

In [4]:

# Charge ta réponse JSON (par exemple si tu l'as dans un fichier 'response.json')
with open('response.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Extraire la liste des records
records = data['pageProps']['records']

# Transformer en DataFrame pandas
df = pd.DataFrame(records)


df_active_drivers = df[df['active'] == True].copy()
# print(df_active_drivers.head())


colonnes_choisies = ["name","starts", "wins","poles", "points","podiums"]       
df_saison_25 = df_active_drivers[colonnes_choisies]
print(df_saison_25)

                      name  starts  wins  poles  points  podiums
33         Alexander Albon     113     0      0   282.0        2
51   Andrea Kimi Antonelli       9     0      0    48.0        0
144           Carlos Sainz     215     4      6  1284.5       27
150        Charles Leclerc     156     8     26  1524.0       46
278           Esteban Ocon     165     1      0   465.0        4
289        Fernando Alonso     410    32     22  2339.0      106
295       Franco Colapinto      12     0      0     5.0        0
312      Gabriel Bortoleto       9     0      0     0.0        0
331         George Russell     137     3      6   825.0       19
425           Isack Hadjar       8     0      0    21.0        0
596           Lance Stroll     174     0      1   306.0        3
597           Lando Norris     137     6     11  1183.0       34
612         Lewis Hamilton     365   105    104  4933.5      202
613            Liam Lawson      20     0      0    10.0        0
673         Max Verstappe

#### Apres avoir chercher en ligne, nous avons la liste de pilotes et leur salaire pour la saison actuelle.

In [5]:
import pandas as pd

Salary = {
    "name": ["Max Verstappen","Lewis Hamilton","Charles Leclerc","Fernando Alonso","Lando Norris","George Russell","Carlos Sainz Jr.","Pierre Gasly","Alex Albon",
        "Nico Hulkenberg","Esteban Ocon","Oscar Piastri","Lance Stroll","Gabriel Bortoleto","Yuki Tsunoda","Kimi Antonelli","Oliver Bearman","Liam Lawson","Jack Doohan","Isack Hadjar"
        ],
    "Base Salary (M $)": [65,60,34,20,20,15,10,10,5,5,5,6,3,2,3,3,1,1,2,1]
}
df_base_salary = pd.DataFrame(Salary)
saison_salary_25 = pd.merge(df_saison_25, df_base_salary, on="name", how="inner")
print(saison_salary_25)

                 name  starts  wins  poles  points  podiums  Base Salary (M $)
0     Charles Leclerc     156     8     26  1524.0       46                 34
1        Esteban Ocon     165     1      0   465.0        4                  5
2     Fernando Alonso     410    32     22  2339.0      106                 20
3   Gabriel Bortoleto       9     0      0     0.0        0                  2
4      George Russell     137     3      6   825.0       19                 15
5        Isack Hadjar       8     0      0    21.0        0                  1
6        Lance Stroll     174     0      1   306.0        3                  3
7        Lando Norris     137     6     11  1183.0       34                 20
8      Lewis Hamilton     365   105    104  4933.5      202                 60
9         Liam Lawson      20     0      0    10.0        0                  1
10     Max Verstappen     218    65     43  3160.5      116                 65
11     Oliver Bearman      12     0      0    13.0  

#### On merge les deux data frame et on comment a entrainer le model. On seppare le data en un set de test et train pour entrainer notre model et le tester sur 20% du set entier.

In [16]:
# Séparer les variables
X = saison_salary_25[['starts', 'wins', 'poles', 'points', 'podiums']]
y = saison_salary_25['Base Salary (M $)']



# Normalisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split en train / test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.1, random_state=46)

# Création du modèle TensorFlow
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])


model.compile(optimizer='adam', loss='mean_squared_error')

# Entraînement
model.fit(X_train, y_train, epochs=100, validation_split=0.2 , verbose=0)

# Évaluation
loss = model.evaluate(X_test, y_test)
print(f"Test Loss (MSE): {loss:.2f}")

# Prédiction sur X_test
predictions = model.predict(X_test, verbose=0)
for i in range(len(predictions)):
    print(f"Actual: {y_test.values[i]}M, Predicted: {predictions[i][0]:.2f}M")

1/1 [==============================] - 0s 22ms/step - loss: 1.5628
Test Loss (MSE): 1.56
Actual: 5M, Predicted: 4.74M
Actual: 3M, Predicted: 4.75M


### Interpreter:

#### Apres avoir recuperer notre model je voudrais voir quelle est le poids de chaque categories et j aimerais voir si des insights interessant peuvent ressortir.

In [15]:
import numpy as np

# weights shape: (features, neurones)
avg_importance_per_feature = np.mean(np.abs(weights), axis=1)

# for i, importance in enumerate(avg_importance_per_feature):
#     print(f"Feature {i} importance moyenne absolue : {importance:.4f}")

features = ['starts', 'wins', 'poles', 'points', 'podiums']
for feature, importance in zip(features, avg_importance_per_feature):
    print(f"{feature}: {importance:.4f}")



starts: 0.1748
wins: 0.1899
poles: 0.1839
points: 0.1619
podiums: 0.1942
